# API Center ❤️ all APIs

## Import the Banking Industry Architecture Network (BIAN) APIs
![image](../../images/import-bian.png)

Playground to experiment importing BIAN APIs from the GitHub repo.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Prerequisites
- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)
- Install [Spectral CLI](https://github.com/stoplightio/spectral)


### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$sourceRepo = "https://github.com/bian-official/public"
$maxAPIsToImport = 10

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "westeurope"
$apicResourceNamePrefix = "apic"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}

# the following metadata will be created during the deployment
$metadata = @(
)

# the following environment(s) will be created during the deployment
$environments = @(
)


Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

### 4️⃣ Clone the repo

Creates a local copy of the specific repository or branch within a repository.


In [ ]:
git clone $sourceRepo ".temp"

### 5️⃣ Crawl APIs and register each one of them in API Center

[Registers](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-register) each valid API in the cloned repo.

In [ ]:
$results = @()

$files = Get-ChildItem -Path ".temp\release12.0.0\semantic-apis\oas3\yamls" -Recurse -File

$counter = 0
foreach ($file in $files) {
    if ($file.Extension -eq ".yaml" || $file.Extension -eq ".yml" || $file.Extension -eq ".json") {
        $fileSize = $file.Length
        $filePath = $file.FullName
        $lintintProblemsJson = spectral lint $filePath --ruleset spectral.yaml -f json             
        $lintintProblems = $lintintProblemsJson | ConvertFrom-Json
        $lintingErrors = 0
        $lintingWarnings = 0
        foreach ($problem in $lintintProblems) {
            if ($problem.severity -eq 0) {
                $lintingErrors++
            } elseif ($problem.severity -eq 1) {
                $lintingWarnings++
            }
        }
        Write-Output "👉🏻 Registering: $filePath"
        $registerOutput = az apic api register -g $resourceGroupName -n $apicResourceName -l $filePath 2>&1
        $registeredAPI = [ordered] @{            
                        Name = $file.Name              
                        File = $file.FullName              
                        Extension = $file.Extension             
                        Size = $file.Length
                        Errors = $lintingErrors
                        Warnings = $lintingWarnings
                        API = if ($registerOutput -match "API was created successfully") {"OK"} else {"ERROR"}
                        Version = if ($registerOutput -match "API version was created successfully") {"OK"} else {"ERROR"}
                        Definition = if ($registerOutput -match "API specification was created successfully") {"OK"} else {"ERROR"}
                        Output = $registerOutput | Out-String
        }    
        $results += New-Object PSObject -Property $registeredAPI
        $counter++
        if ($counter -ge $maxAPIsToImport) {
            Write-Output "🚫 Max APIs to import reached. Done for now."
            break
        }
    }
}
Write-Output "✅ $counter APIs imported."

$results | export-csv -Path .temp/result.csv -NoTypeInformation -Delimiter ";"


### 7️⃣ Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [ ]:
az apic api list -g $resourceGroupName -n $apicResourceName --query "[].{Name:name, Title:title, ContactEmail:contacts[0].email}" -o table

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.